*   Neural Netwroks & Deep Learning - Dr Ghiasi

*   Mohammad Parsa Etemadheravi
*   9812762441

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In this exercise we tend to predict the age of individuals with respect to the image of their face by transfer learnign.

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import torch.backends.cudnn as cudnn
from torch.autograd import Variable
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
from PIL import Image
from tempfile import TemporaryDirectory
from torchvision import models


In [3]:
# Transform on data
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
        transforms.ColorJitter(),
        transforms.RandomVerticalFlip()
           ]),
    'val': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])

    ]),
}

# getting data
data_dir = '/content/drive/My Drive/NN-Course/AgeDetectionDataset'
image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x),
                                          data_transforms[x])
                  for x in ['train', 'val']}
dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=4,
                                             shuffle=True, num_workers=4)
              for x in ['train', 'val']}
dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val']}
class_names = image_datasets['train'].classes

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


In [ ]:
# class NeuralNetwork(nn.Module) :

#   def __init__(self):
#     super().__init__()

# #     write your code

#   def forward(self, x ):
# #     write your code

In [4]:
def train_model(model, criterion, optimizer, scheduler, num_epochs=25):
    since = time.time()

    # Create a temporary directory to save training checkpoints
    with TemporaryDirectory() as tempdir:
        best_model_params_path = os.path.join(tempdir, 'best_model_params.pt')

        torch.save(model.state_dict(), best_model_params_path)
        best_acc = 0.0

        for epoch in range(num_epochs):
            print(f'Epoch {epoch}/{num_epochs - 1}')
            print('-' * 10)

            # Each epoch has a training and validation phase
            for phase in ['train', 'val']:
                if phase == 'train':
                    model.train()  # Set model to training mode
                else:
                    model.eval()   # Set model to evaluate mode

                running_loss = 0.0
                running_corrects = 0

                # Iterate over data.
                for inputs, labels in dataloaders[phase]:
                    inputs = inputs.to(device)
                    labels = labels.to(device)

                    # zero the parameter gradients
                    optimizer.zero_grad()

                    # forward
                    # track history if only in train
                    with torch.set_grad_enabled(phase == 'train'):
                        outputs = model(inputs)
                        _, preds = torch.max(outputs, 1)
                        loss = criterion(outputs, labels)

                        # backward + optimize only if in training phase
                        if phase == 'train':
                            loss.backward()
                            optimizer.step()

                    # statistics
                    running_loss += loss.item() * inputs.size(0)
                    running_corrects += torch.sum(preds == labels.data)
                if phase == 'train':
                    scheduler.step()

                epoch_loss = running_loss / dataset_sizes[phase]
                epoch_acc = running_corrects.double() / dataset_sizes[phase]

                print(f'{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')

                # deep copy the model
                if phase == 'val' and epoch_acc > best_acc:
                    best_acc = epoch_acc
                    torch.save(model.state_dict(), best_model_params_path)

            print()

        time_elapsed = time.time() - since
        print(f'Training complete in {time_elapsed // 60:.0f}m {time_elapsed % 60:.0f}s')
        print(f'Best val Acc: {best_acc:4f}')

        # load best model weights
        model.load_state_dict(torch.load(best_model_params_path))
    return model

## **Implementing VGG16:** Best Validation Accuracy for 10 epochs: **32%**


In [13]:
class NeuralNetwork(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(NeuralNetwork, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, num_classes)

    def forward(self, x):
        x = x.view(x.size(0), -1)
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        return x


In [14]:
class TransferLearningModel(nn.Module):
    def __init__(self, vgg16_model, mlp_model):
        super(TransferLearningModel, self).__init__()
        self.vgg16 = vgg16_model
        self.mlp = mlp_model

        # Remove the last fully connected layer of VGG16
        vgg16_classifier = list(self.vgg16.classifier.children())[:-1]
        self.vgg16.classifier = nn.Sequential(*vgg16_classifier)
        mlp_input_size = 4096  # Adjust this based on the VGG16 classifier's last layer output size
        self.mlp.fc1 = nn.Linear(mlp_input_size, hidden_size)

    def forward(self, x):
        vgg16_features = self.vgg16.features(x)
        vgg16_avgpool = self.vgg16.avgpool(vgg16_features)
        # print(x.size(0))
        vgg16_flattened = vgg16_avgpool.view(x.size(0), -1)
        # vgg16_flattened = F.relu(nn.Linear(512 * 7 * 7, hidden_size)(vgg16_flattened))
        vgg16_output = self.vgg16.classifier(vgg16_flattened)

        # Forward pass through the adjusted MLP
        mlp_output = self.mlp(vgg16_output)
        combined_output = torch.cat((vgg16_output, mlp_output), dim=1)
        return combined_output

In [17]:
hidden_size = 1000
num_classes = 5



# Create instances of the models
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")


vgg16_model = models.vgg16(pretrained = 'IMAGENET1K_V1')
mlp_model = NeuralNetwork(input_size=hidden_size, hidden_size=hidden_size, num_classes=num_classes)
transfer_learning_model = TransferLearningModel(vgg16_model, mlp_model)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
transfer_learning_model = transfer_learning_model.to(device)



In [16]:
loss = nn.CrossEntropyLoss()
optimizer = optim.SGD(params=transfer_learning_model.parameters(), lr=0.001)
exp_lr_scheduler = lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)

train_model(transfer_learning_model, loss, optimizer, exp_lr_scheduler, num_epochs=10)


Epoch 0/9
----------
train Loss: 8.5810 Acc: 0.0000
val Loss: 7.8341 Acc: 0.1200

Epoch 1/9
----------
train Loss: 7.2656 Acc: 0.1600
val Loss: 6.0140 Acc: 0.2400

Epoch 2/9
----------
train Loss: 6.9872 Acc: 0.1920
val Loss: 7.0551 Acc: 0.3200

Epoch 3/9
----------
train Loss: 6.6969 Acc: 0.2240
val Loss: 6.5985 Acc: 0.2400

Epoch 4/9
----------
train Loss: 6.6037 Acc: 0.2000
val Loss: 6.0820 Acc: 0.1200

Epoch 5/9
----------
train Loss: 6.8583 Acc: 0.1920
val Loss: 6.1214 Acc: 0.2400

Epoch 6/9
----------
train Loss: 6.7198 Acc: 0.2240
val Loss: 7.1835 Acc: 0.2400

Epoch 7/9
----------
train Loss: 7.1116 Acc: 0.1760
val Loss: 6.9124 Acc: 0.2400

Epoch 8/9
----------
train Loss: 6.6976 Acc: 0.2400
val Loss: 6.6048 Acc: 0.2400

Epoch 9/9
----------
train Loss: 6.5284 Acc: 0.2400
val Loss: 6.5108 Acc: 0.2000

Training complete in 3m 14s
Best val Acc: 0.320000


TransferLearningModel(
  (vgg16): VGG(
    (features): Sequential(
      (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): ReLU(inplace=True)
      (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (3): ReLU(inplace=True)
      (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (6): ReLU(inplace=True)
      (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (8): ReLU(inplace=True)
      (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (11): ReLU(inplace=True)
      (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (13): ReLU(inplace=True)
      (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (15): ReLU(inplace=True

## **Implementing RESNet18:** Best Validation Accuracy for 10 epochs: **40%**


In [ ]:
class ModifiedResNet18(nn.Module):
    def __init__(self, num_classes):
        super(ModifiedResNet18, self).__init__()
        resnet18_model = models.resnet18(pretrained=True)

        # Remove the original fully connected layer
        self.resnet18 = nn.Sequential(*list(resnet18_model.children())[:-1])

    def forward(self, x):
        x = self.resnet18(x)
        x = x.view(x.size(0), -1)
        return x

In [ ]:
class TransferLearningModel(nn.Module):
    def __init__(self, resnet18_model, mlp_model):
        super(TransferLearningModel, self).__init__()
        self.resnet18 = resnet18_model
        self.mlp = mlp_model

    def forward(self, x):
        x = self.resnet18(x)
        x = self.mlp(x)
        return x


In [ ]:
class NeuralNetwork(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(NeuralNetwork, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, num_classes)

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        return x

In [ ]:
input_size = 512  # ResNet18 outputs 512 features
hidden_size = 256
num_classes = 5


resnet18_model = ModifiedResNet18(num_classes=num_classes)
mlp_model = NeuralNetwork(input_size=input_size, hidden_size=hidden_size, num_classes=num_classes)
transfer_learning_model = TransferLearningModel(resnet18_model, mlp_model)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
transfer_learning_model = transfer_learning_model.to(device)

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 165MB/s]


In [ ]:
loss = nn.CrossEntropyLoss()
optimizer = optim.SGD(params=transfer_learning_model.parameters(), lr=0.001)
exp_lr_scheduler = lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)

train_model(transfer_learning_model, loss, optimizer, exp_lr_scheduler, num_epochs=10)

Epoch 0/9
----------
train Loss: 1.6402 Acc: 0.1840
val Loss: 1.6267 Acc: 0.3600

Epoch 1/9
----------
train Loss: 1.6125 Acc: 0.2080
val Loss: 1.6217 Acc: 0.4000

Epoch 2/9
----------
train Loss: 1.6013 Acc: 0.2400
val Loss: 1.6217 Acc: 0.3200

Epoch 3/9
----------
train Loss: 1.6018 Acc: 0.2160
val Loss: 1.6017 Acc: 0.3200

Epoch 4/9
----------
train Loss: 1.5952 Acc: 0.2240
val Loss: 1.6133 Acc: 0.2800

Epoch 5/9
----------
train Loss: 1.5903 Acc: 0.3040
val Loss: 1.6076 Acc: 0.2800

Epoch 6/9
----------
train Loss: 1.5882 Acc: 0.2960
val Loss: 1.6108 Acc: 0.2400

Epoch 7/9
----------
train Loss: 1.5802 Acc: 0.2320
val Loss: 1.6162 Acc: 0.2400

Epoch 8/9
----------
train Loss: 1.5753 Acc: 0.2320
val Loss: 1.6115 Acc: 0.2400

Epoch 9/9
----------
train Loss: 1.5808 Acc: 0.2640
val Loss: 1.6087 Acc: 0.2400

Training complete in 3m 9s
Best val Acc: 0.400000


TransferLearningModel(
  (resnet18): ModifiedResNet18(
    (resnet18): Sequential(
      (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU(inplace=True)
      (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
      (4): Sequential(
        (0): BasicBlock(
          (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (relu): ReLU(inplace=True)
          (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        )
        (1): BasicBlock(
          (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn1): 

## **Implementing RESNet50:** Best Validation Accuracy for 10 epochs: **24%**

In [18]:
class ModifiedResNet50(nn.Module):
    def __init__(self, num_classes):
        super(ModifiedResNet50, self).__init__()
        # Load pre-trained ResNet50 model
        resnet50_model = models.resnet50(pretrained=True)

        # Remove the original fully connected layer
        self.resnet50 = nn.Sequential(*list(resnet50_model.children())[:-1])

    def forward(self, x):
        x = self.resnet50(x)
        x = x.view(x.size(0), -1)
        return x

In [19]:
class TransferLearningModel(nn.Module):
    def __init__(self, resnet50_model, mlp_model):
        super(TransferLearningModel, self).__init__()
        self.resnet50 = resnet50_model
        self.mlp = mlp_model

    def forward(self, x):
        x = self.resnet50(x)
        x = self.mlp(x)
        return x

In [22]:
input_size = 2048  # ResNet50 outputs 2048 features
hidden_size = 256  # Adjust as needed
num_classes = 5  # Adjust based on the number of age classes

# Create instances of the models
resnet50_model = ModifiedResNet50(num_classes=num_classes)
mlp_model = NeuralNetwork(input_size=input_size, hidden_size=hidden_size, num_classes=num_classes)
transfer_learning_model = TransferLearningModel(resnet50_model, mlp_model)

# Move the model to the GPU if available
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
transfer_learning_model = transfer_learning_model.to(device)


In [23]:
# Define the loss function, optimizer, and learning rate scheduler
loss = nn.CrossEntropyLoss()
optimizer = optim.SGD(params=transfer_learning_model.parameters(), lr=0.001)
exp_lr_scheduler = lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)

# Train the model using the provided training function
train_model(transfer_learning_model, loss, optimizer, exp_lr_scheduler, num_epochs=10)

Epoch 0/9
----------
train Loss: 1.6184 Acc: 0.1840
val Loss: 1.6192 Acc: 0.2400

Epoch 1/9
----------
train Loss: 1.6156 Acc: 0.2080
val Loss: 1.6177 Acc: 0.2000

Epoch 2/9
----------
train Loss: 1.6129 Acc: 0.1840
val Loss: 1.6203 Acc: 0.1200

Epoch 3/9
----------
train Loss: 1.6060 Acc: 0.2160
val Loss: 1.6168 Acc: 0.1600

Epoch 4/9
----------
train Loss: 1.6099 Acc: 0.2080
val Loss: 1.6168 Acc: 0.1600

Epoch 5/9
----------
train Loss: 1.5975 Acc: 0.2240
val Loss: 1.6165 Acc: 0.1200

Epoch 6/9
----------
train Loss: 1.6005 Acc: 0.2640
val Loss: 1.6128 Acc: 0.1200

Epoch 7/9
----------
train Loss: 1.5896 Acc: 0.3040
val Loss: 1.6110 Acc: 0.1200

Epoch 8/9
----------
train Loss: 1.5850 Acc: 0.3280
val Loss: 1.6155 Acc: 0.0800

Epoch 9/9
----------
train Loss: 1.5936 Acc: 0.2880
val Loss: 1.6134 Acc: 0.1200

Training complete in 3m 13s
Best val Acc: 0.240000


TransferLearningModel(
  (resnet50): ModifiedResNet50(
    (resnet50): Sequential(
      (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU(inplace=True)
      (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
      (4): Sequential(
        (0): Bottleneck(
          (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (relu): ReLU